In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nioro,GM,2021-02-07 16:49:30,13.3500,-15.7500,94.5,10,46,11.59
1,1,Port Hardy,CA,2021-02-07 16:47:21,50.6996,-127.4199,39.2,93,75,5.75
2,2,Kodiak,US,2021-02-07 16:47:24,57.7900,-152.4072,26.6,63,1,4.61
3,3,Sitka,US,2021-02-07 16:49:30,57.0531,-135.3300,28.4,64,75,3.44
4,4,Yellowknife,CA,2021-02-07 16:47:20,62.4560,-114.3525,-49.0,65,1,4.61


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# # Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     temps.append(max(temp, 0))

In [6]:
# # 1. Assign the locations to an array of latitude and longitude pairs.
# locations = [latitude, longitude]
# # 2. Assign the weights variable to some values.
# temperatures = # an array of length equal to the locations array length
# # 3. Assign the figure variable to the gmaps.figure() attribute.
# fig = gmaps.figure()
# # 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# # 5. Add the heatmap layer.
# fig.add_layer(heatmap_layer)
# # 6. Call the figure to plot the data.
# fig
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
# fig = gmaps.figure()
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
# heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent Cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Rikitea,PF,2021-02-07 16:44:50,-23.1203,-134.9692,77.27,72,95,11.77
13,13,Georgetown,MY,2021-02-07 16:49:31,5.4112,100.3354,82.40,83,20,3.44
14,14,Mahebourg,MU,2021-02-07 16:46:09,-20.4081,57.7000,80.60,78,20,10.36
18,18,Tevaitoa,PF,2021-02-07 16:49:32,-16.7833,-151.5000,81.00,75,5,13.11
22,22,Puerto Ayora,EC,2021-02-07 16:47:20,-0.7393,-90.3518,84.20,70,0,18.41
25,25,Cururupu,BR,2021-02-07 16:49:33,-1.8283,-44.8683,87.89,59,48,7.40
33,33,Vao,NC,2021-02-07 16:49:33,-22.6667,167.4833,77.14,86,11,3.69
35,35,Bambous Virieux,MU,2021-02-07 16:47:18,-20.3428,57.7575,81.00,78,20,10.36
37,37,Arraial Do Cabo,BR,2021-02-07 16:49:34,-22.9661,-42.0278,77.00,83,75,9.22
40,40,San Cristobal,VE,2021-02-07 16:47:15,7.7669,-72.2250,86.00,54,20,11.50


In [12]:
# preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Rikitea,PF,77.27,-23.1203,-134.9692,
13,Georgetown,MY,82.40,5.4112,100.3354,
14,Mahebourg,MU,80.60,-20.4081,57.7000,
18,Tevaitoa,PF,81.00,-16.7833,-151.5000,
22,Puerto Ayora,EC,84.20,-0.7393,-90.3518,
25,Cururupu,BR,87.89,-1.8283,-44.8683,
33,Vao,NC,77.14,-22.6667,167.4833,
35,Bambous Virieux,MU,81.00,-20.3428,57.7575,
37,Arraial Do Cabo,BR,77.00,-22.9661,-42.0278,
40,San Cristobal,VE,86.00,7.7669,-72.2250,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
#     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
#     Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):

        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Rikitea,PF,77.27,-23.1203,-134.9692,Pension Maro'i
13,Georgetown,MY,82.40,5.4112,100.3354,Cititel Penang
14,Mahebourg,MU,80.60,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
18,Tevaitoa,PF,81.00,-16.7833,-151.5000,Hôtel Raiatea Lodge
22,Puerto Ayora,EC,84.20,-0.7393,-90.3518,Castillo Galapagos
25,Cururupu,BR,87.89,-1.8283,-44.8683,Pousada Través'cia
33,Vao,NC,77.14,-22.6667,167.4833,Hotel Kou-Bugny
35,Bambous Virieux,MU,81.00,-20.3428,57.7575,Casa Tia Villa
37,Arraial Do Cabo,BR,77.00,-22.9661,-42.0278,Pousada Porto Praia
40,San Cristobal,VE,86.00,7.7669,-72.2250,Pirineos


In [16]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [18]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))